# Storm surge level

Notebook environment to migrate netcdf files to zarr and geojson

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive, rel_root
from etl.extract import clear_zarr_information, get_geojson
from etl.keys import load_env_variables, load_google_credentials

# Define both local and remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

C:\Users\kras\Anaconda3\envs\bathymetry_env\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


<IPython.core.display.Javascript object>

In [14]:
# Project paths & files (manual input)
dataset_dir = coclico_data_dir.joinpath("01_storm_surge_jrc")
dataset_historical_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_Historical.nc")
dataset_rcp45_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_RCP45.nc")
dataset_rcp85_path = dataset_dir.joinpath("CoastAlRisk_Europe_EESSL_RCP85.nc")
dataset_out_file = "CoastAlRisk_Europe_EESSL"

<IPython.core.display.Javascript object>

### load environment variables and set credentials

In [4]:
load_env_variables(env_var_keys=["MAPBOX_ACCESS_TOKEN"])
load_google_credentials(
    google_token=coclico_data_dir.joinpath("google_credentials.json")
)

Environmental variables loaded.
Google Application Credentials load into environment.


C:\SnapVolumesTemp\MountPoints\{45c63495-0000-0000-0000-100000000000}\{D2C8ADC5-14BE-4A35-B9CA-C9C2D1930AB1}\SVROOT\Users\kras\Documents\GitHub\coclicodata\etl\keys.py:33: CredentialLeakageWarning: Keys loaded from shared network drive.
  warnings.warn("Keys loaded from shared network drive.", CredentialLeakageWarning)


<IPython.core.display.Javascript object>

### write data to Zarr files (dataset dependent)

In [5]:
# open datasets
dataset_historical = xr.open_dataset(dataset_historical_path)
dataset_45rcp = xr.open_dataset(dataset_rcp45_path)
dataset_85rcp = xr.open_dataset(dataset_rcp85_path)

# check original dataset
# dataset_historical

C:\Users\kras\Anaconda3\envs\bathymetry_env\lib\site-packages\xarray\backends\plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<IPython.core.display.Javascript object>

In [6]:
# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_historical = dataset_historical.rename_dims({"row": "stations"})
dataset_45rcp = dataset_45rcp.rename_dims({"row": "stations"})
dataset_85rcp = dataset_85rcp.rename_dims({"row": "stations"})
dataset_historical = dataset_historical.swap_dims({"col": "RP"})
dataset_45rcp = dataset_45rcp.swap_dims({"col": "RP"})
dataset_85rcp = dataset_85rcp.swap_dims({"col": "RP"})

# rename variables, if necessary
# dataset_historical = dataset_historical.rename_vars({"RP": "rp"})
# dataset_45rcp = dataset_45rcp.rename_vars({"RP": "rp"})
# dataset_85rcp = dataset_85rcp.rename_vars({"RP": "rp"})

# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_historical = dataset_historical.set_coords(["longitude", "latitude", "RP"])
dataset_45rcp = dataset_45rcp.set_coords(["longitude", "latitude", "RP"])
dataset_85rcp = dataset_85rcp.set_coords(["longitude", "latitude", "RP"])

<IPython.core.display.Javascript object>

In [7]:
# concat datasets along new dimension with index values and name derived from pandas index object, if necessary
dataset = xr.concat(
    [dataset_historical, dataset_45rcp, dataset_85rcp],
    pd.Index(["Historical", "RCP45", "RCP85"], name="scenario"),
)

<IPython.core.display.Javascript object>

In [8]:
# re-order shape of the data variables
dataset = dataset.transpose("scenario", "stations", "RP")

<IPython.core.display.Javascript object>

In [34]:
# add attributes to variables and coordinates, if necessary

# specify custom (CF convention) attributes
dataset_attributes = {
    "geometry": {
        "long_name": "nuts 2 polygons in wkt string (degrees; epgsg:4326)",
        "geometry_type": "polygon",
        "units": "degrees",
        "comment": "Describe nuts regions (2010 version), which are available from Eurostat.",
        "crs_wkt": f"..",
    },
    "layer": {"long_name": "socio-economic layer of the dataset"},
    "nuts": {
        "long_name": "nuts region acronym",
        "source": "Nuts regions (2010 version), available from Eurostat.",
    },
    "name": {
        "long_name": "nuts region name",
        "source": "Nuts regions (2010 version), available from Eurostat.",
    },
    "country": {"long_name": "country acronym"},
}

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        dataset[k].attrs = dataset_attributes[k]
    except:
        continue

<IPython.core.display.Javascript object>

In [36]:
dataset

<xarray.Dataset>
Dimensions:    (stations: 2242, scenario: 3, RP: 8)
Coordinates:
    longitude  (stations) float64 -0.1 -0.1 -0.1 -0.1 -0.3 ... 9.9 9.9 9.9 9.9
    latitude   (stations) float64 36.1 39.3 49.7 54.3 ... 57.7 58.7 64.5 64.7
  * RP         (RP) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
  * scenario   (scenario) object 'Historical' 'RCP45' 'RCP85'
Dimensions without coordinates: stations
Data variables:
    ssl        (scenario, stations, RP) float64 1.024 1.051 ... 2.712 2.805
Attributes:
    title:            European extreme storm surge level
    Institution:      Joint European Research Center, Institute of Environmen...
    Project Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project Acronym:  CoastAlRisk
    reference:        Vousdoukas MI, Voukouvalas E, Annunziato A, Giardino A,...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...

<IPython.core.display.Javascript object>

In [9]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

dataset

<xarray.Dataset>
Dimensions:    (stations: 2242, scenario: 3, RP: 8)
Coordinates:
    longitude  (stations) float64 -0.1 -0.1 -0.1 -0.1 -0.3 ... 9.9 9.9 9.9 9.9
    latitude   (stations) float64 36.1 39.3 49.7 54.3 ... 57.7 58.7 64.5 64.7
  * RP         (RP) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
  * scenario   (scenario) object 'Historical' 'RCP45' 'RCP85'
Dimensions without coordinates: stations
Data variables:
    ssl        (scenario, stations, RP) float64 1.024 1.051 ... 2.712 2.805
Attributes:
    title:            European extreme storm surge level
    Institution:      Joint European Research Center, Institute of Environmen...
    Project Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project Acronym:  CoastAlRisk
    reference:        Vousdoukas MI, Voukouvalas E, Annunziato A, Giardino A,...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.0
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...

<IPython.core.display.Javascript object>

In [15]:
# export to zarr in write mode (to overwrite if exists)
dataset.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [24]:
dataset["ssl"]

<xarray.DataArray 'ssl' (scenario: 3, stations: 2242, RP: 8)>
array([[[1.02407, 1.0509 , 1.0778 , ..., 1.16939, 1.20762, 1.23753],
        [1.24336, 1.28508, 1.32805, ..., 1.4857 , 1.55811, 1.61812],
        [1.95   , 2.08892, 2.20901, ..., 2.51257, 2.60544, 2.66859],
        ...,
        [2.05624, 2.20418, 2.33512, ..., 2.67975, 2.78853, 2.86284],
        [1.86446, 2.02245, 2.16554, ..., 2.55166, 2.67389, 2.7566 ],
        [1.87579, 2.02328, 2.1549 , ..., 2.49943, 2.60478, 2.67487]],

       [[1.03165, 1.05851, 1.08478, ..., 1.16899, 1.20167, 1.22622],
        [1.23326, 1.27602, 1.32215, ..., 1.50603, 1.59589, 1.67189],
        [1.97503, 2.12185, 2.25063, ..., 2.58303, 2.68587, 2.75567],
        ...,
        [2.11645, 2.25912, 2.38164, ..., 2.68137, 2.7671 , 2.82265],
        [1.91546, 2.07808, 2.22304, ..., 2.60134, 2.71649, 2.79291],
        [1.9071 , 2.05876, 2.19542, ..., 2.56052, 2.67487, 2.75187]],

       [[1.03412, 1.06436, 1.09628, ..., 1.21896, 1.27761, 1.32699],
        [1.22974, 1.2753 , 1.32831, ..., 1.5973 , 1.77785, 1.96619],
        [1.985  , 2.12493, 2.24118, ..., 2.50843, 2.57984, 2.62478],
        ...,
        [2.10311, 2.24885, 2.37682, ..., 2.70514, 2.80472, 2.87116],
        [1.92381, 2.08563, 2.23729, ..., 2.67873, 2.83175, 2.94024],
        [1.89731, 2.04812, 2.18738, ..., 2.58059, 2.7125 , 2.80462]]])
Coordinates:
    longitude  (stations) float64 -0.1 -0.1 -0.1 -0.1 -0.3 ... 9.9 9.9 9.9 9.9
    latitude   (stations) float64 36.1 39.3 49.7 54.3 ... 57.7 58.7 64.5 64.7
  * RP         (RP) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
  * scenario   (scenario) object 'Historical' 'RCP45' 'RCP85'
Dimensions without coordinates: stations
Attributes:
    long_name:  storm surge level
    units:      m

<IPython.core.display.Javascript object>

In [23]:
dataset["scenario"]

<xarray.DataArray 'scenario' (scenario: 3)>
array(['Historical', 'RCP45', 'RCP85'], dtype=object)
Coordinates:
  * scenario  (scenario) object 'Historical' 'RCP45' 'RCP85'

<IPython.core.display.Javascript object>